In [1]:
import numpy as np
import pandas as pd

In [9]:
my_url = 'file:///C:/Users/WEIL/Documents/GitHub/yonge_eglinton_housing/web_link/Link_20180913.html'
# my_url = "http://v3.torontomls.net/Live/Pages/Public/Link.aspx?Key=781746b718784d229fdca9272f11a25f&App=TREB"

In [10]:
# Connect to url and grab page contents

import urllib.request
with urllib.request.urlopen(my_url) as client:
    web_page = client.read()

# import requests
# web_page = requests.get(my_url).content

### Use XPath with lxml

XPath tutorial: 
https://blog.scrapinghub.com/2016/10/27/an-introduction-to-xpath-with-examples

In [6]:
from lxml import html

In [11]:
parsed_page = html.fromstring(web_page)

In [12]:
containers = parsed_page.xpath('//*[@id="form1"]/div[3]/div[5]/div')
print('Number of Houses:', len(containers))
house_df = pd.DataFrame()
room_info_df = pd.DataFrame()
for container in containers:
    row = {}
    counter = 1
    for field in container.findall('.//div[2]/div[2]/div[1]/div//*[@class="formitem formfield"]'):
        try:
            label = field.find('.//label').text
        except:
            label = counter
            counter += 1
        try:
            value = field.find('.//span').text
        except:
            value = field.find('.//a').text
        # check if duplicated label
        if label not in row.keys():
            row.update({label: value})
        elif row[label] != value:
            label = label + "_"
            row.update({label: value})

    # spin off the room info into a seperate df
    room_info_keys = [j for j in np.sort([i for i in row.keys() if str(i).isdigit()])[15:-7]]
    room_info = {}
    for key in room_info_keys:
        room_info.update({key: row.pop(key)})
    # tabulate the rest info into the house df
    house = pd.DataFrame.from_dict(row, orient='index').T
    # ajust column names of house df
    j_ = 16
    for j in [j for j in np.sort([i for i in row.keys() if str(i).isdigit()])[-7:]]:
        house.rename(columns={j: j_}, inplace=True)
        j_ += 1
    room_info = pd.DataFrame.from_dict(room_info, orient='index').T
    # adjust column names of room_info df
    room_info.columns = [i for i in range(room_info.shape[1])]
    
    house_df = pd.concat([house_df, house], ignore_index=True)
    room_info_df = pd.concat([room_info_df, room_info])


print("house_df.shape:", house_df.shape)
print("room_info_df.shape:", room_info_df.shape)

Number of Houses: 2
house_df.shape: (2, 81)
room_info_df.shape: (2, 77)


In [13]:
house_df.columns

Index([                1,                 2,                 3,
                       4,                 5,           'List:',
                  'For:',                 6,                 7,
                       8,                 9,          'Taxes:',
                      10,           'SPIS:',    'Last Status:',
                      11,                12,                13,
                  'Lot:',          'Irreg:',    'Fronting On:',
                 'Acre:',            'Rms:',       'Bedrooms:',
            'Washrooms:',                14,   'Dir/Cross St:',
                 'MLS#:',     'Possession:',           'PIN#:',
             'Kitchens:',         'Fam Rm:',       'Basement:',
        'Fireplace/Stv:',           'Heat:',            'A/C:',
          'Central Vac:',        'Apx Age:',       'Apx Sqft:',
           'Assessment:',           'POTL:',    'POTL Mo Fee:',
        'Elevator/Lift:',    'Laundry Lev:',   'Phys Hdp-Eqp:',
             'Exterior:',          'Driv

In [14]:
room_info_df.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
            68, 69, 70, 71, 72, 73, 74, 75, 76],
           dtype='int64')

In [15]:
house_df.head(3)

,1,2,3,4,5,List:,For:,6,7,8,...,17,18,19,20,Client Remks:,Extras:,21,Ph:,Fax:,22
0,42 Burnaby Blvd,None,Toronto,Ontario,M4R1B4,"$1,198,000",Sale,Toronto C03,Yonge-Eglinton,Toronto,...,2.64,None,None,None,"Classic North Toronto Home, Central Location, ...","Ductless A/C, Walk Out To Deck, Large Private ...","ROYAL LEPAGE REAL ESTATE SERVICES LTD., BROKERAGE",416-487-4311,416-487-3699,2018
1,571 Oriole Pkwy,None,Toronto,Ontario,M4R2B9,"$1,595,000",Sale,Toronto C03,Yonge-Eglinton,Toronto,...,None,None,None,None,"Located On A Quiet Part Of The Street, Within ...",An Outstanding Professionally Landscaped Backy...,ROYAL LEPAGE REAL ESTATE SERVICES HEAPS ESTRIN...,416-424-4910,416-424-4909,2018


In [16]:
room_info_df

,0,1,2,3,4,5,6,7,8,9,...,67,68,69,70,71,72,73,74,75,76
0,None,None,1,Living,Ground,5.03,3.58,Hardwood Floor,Crown Moulding,Brick Fireplace,...,Rec,Bsmt,4.47,3.61,Ceramic Floor,Combined W/Laundry,None,10,Utility,Bsmt
0,None,None,1,Foyer,Main,2.84,2.54,Hardwood Floor,Closet,Leaded Glass,...,Rec,Lower,3.63,3.00,None,None,None,10,Laundry,Lower


In [18]:
house_df.to_csv(r"C:\Users\WEIL\Documents\GitHub\yonge_eglinton_housing\web_scraping_output\link_20180913_house_df.csv", 
                header=True, sep=',')
room_info_df.to_csv(r"C:\Users\WEIL\Documents\GitHub\yonge_eglinton_housing\web_scraping_output\link_20180913_room_info_df.csv", 
                header=True, sep=',')

### Use BeautifulSoup

Note, it is found in this example that BeautifulSoup did not grap all the objects (maybe because of incorrect keys).

In [38]:
from bs4 import BeautifulSoup

In [39]:
soup = BeautifulSoup(web_page,'lxml')
# soup = BeautifulSoup(web_page,'html.parser')

In [ ]:
###  BeautifulSoup Examples

# title of the page
print(soup.title)

# get attributes:
print(soup.title.name)

# get values:
print(soup.title.string)

# beginning navigation:
print(soup.title.parent.name)

# get all texts
print(soup.get_text())

# nicer printout
print(soup.prettify())

# getting tags/values:
print(soup.p)
print(soup.h1)
print(soup.body)

# soup.find()
soup('span', class_='formitem formfield')   # return a list
soup.find('span', class_='formitem formfield').get_text()  # return soup object
containers = soup.findAll('div', {"class":"link-item status-sld hasheader loaded"})
containers = soup.find_all('div', class_='link-item status-sld hasheader loaded')

for paragraph in soup.find_all('p'):
    print(str(paragraph.text))

for url in soup.find_all('a'):
    print(url.get('href'))

In [41]:
containers = soup.findAll('div', {"class":"link-item status-sld hasheader loaded"})
# containers = soup.find_all('div', class_='link-item status-sld hasheader loaded')
print('Number of Houses:', len(containers))

house_df = pd.DataFrame()
room_info_df = pd.DataFrame()
for container in containers:
    row = {}
    counter = 1
    for item in container.findAll('span', class_='formitem formfield'):
        try:
            label = item.label
            label = label.get_text()
        except:
            label = counter
            counter += 1
        try:
            value = item.find('span', class_='value')
            value = value.get_text()
        except:
            value = item.find('a', class_='formlink value')
            value = value.get_text()
        # check if duplicated label
        if label not in row.keys():
            row.update({label: value})
        elif row[label] != value:
            label = label + "_"
            row.update({label: value})

    # spin off the room info into a seperate df
    room_info_keys = [j for j in np.sort([i for i in row.keys() if str(i).isdigit()])[15:-7]]
    room_info = {}
    for key in room_info_keys:
        room_info.update({key: row.pop(key)})
    # tabulate the rest info into the house df
    house = pd.DataFrame.from_dict(row, orient='index').T
    # ajust column names of house df
    j_ = 16
    for j in [j for j in np.sort([i for i in row.keys() if str(i).isdigit()])[-7:]]:
        house.rename(columns={j: j_}, inplace=True)
        j_ += 1
    room_info = pd.DataFrame.from_dict(room_info, orient='index').T
    # adjust column names of room_info df
    room_info.columns = [i for i in range(room_info.shape[1])]
    
    house_df = pd.concat([house_df, house], ignore_index=True)
    room_info_df = pd.concat([room_info_df, room_info])

print("house_df.shape:", house_df.shape)
print("room_info_df.shape:", room_info_df.shape)

Number of Houses: 3
house_df.shape: (3, 94)
room_info_df.shape: (3, 97)


In [42]:
house_df.head()

,1,2,3,4,5,Sold:,List:,6,7,8,...,21,22,Contract Date:,Expiry Date:,Last Update:,Sold Date:,Closing Date:,CB Comm:,Leased Terms:,Original:
0,168 Hillsdale Ave E,,Toronto,Ontario,M4S1T5,"$1,310,000","$1,329,000",Toronto C10,Mount Pleasant West,Toronto,...,"Spencer Eyman, Salesperson",,5/28/2018,8/30/2018,6/11/2018,6/08/2018,9/11/2018,2.5%,,"$1,329,000"
1,218 Hillsdale Ave E,,Toronto,Ontario,M4S 1T6,"$1,425,000","$1,449,000",Toronto C10,Mount Pleasant West,Toronto,...,"Kevin Kilislian, Salesperson",,6/21/2018,8/31/2018,6/26/2018,6/23/2018,7/27/2018,2.5 % + Hst,,"$1,449,000"
2,17 Oswald Cres,,Toronto,Ontario,M4S2H5,"$1,600,000","$1,695,000",Toronto C10,Mount Pleasant West,Toronto,...,"Bernard W. K. Tsui, Broker",,5/22/2018,8/22/2018,6/13/2018,6/12/2018,7/31/2018,2.5% + Hst,,"$1,695,000"


In [43]:
room_info_df.head()

,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
0,,,1,Living,Ground,3.16,3.15,Hardwood Floor,Fireplace,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,,,1,Living,Main,6.55,3.10,Combined W/Family,Hardwood Floor,Bay Window,...,,,,12,Bathroom,Lower,,,3 Pc Bath,Ceramic Floor
0,,,1,Foyer,Main,3.63,2.11,Hardwood Floor,Closet,,...,Laminate,Laundry Sink,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
